# Portfolio Optimization using Multiple Regression

### Imports

In [42]:
import warnings

In [43]:
warnings.filterwarnings('ignore') 

In [44]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib as plt
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Data Collection

Randomly chosen 100 tickers

In [52]:
tickers = ["META", "GOOGL", "NFLX", "DIS", "CMCSA", "CHTR", "T", "VZ", 
            "AMZN", "TSLA", "HD", "NKE", "MCD", "SBUX", "TGT", "LOW", "GM", "F", 
            "PG", "WMT", "KO", "PEP", "COST", "CL", "MDLZ", "KHC", "PM", "MO", 
            "XOM", "CVX", "COP", "SLB", "KMI", "OXY", "PSX", "MPC", "EOG", 
            "JPM", "BRK.B", "BAC", "WFC", "C", "GS", "MS", "AXP", "V", "MA", 
            "JNJ", "PFE", "MRK", "UNH", "ABT", "AMGN", "BMY", "MDT", "TMO", 
            "BA", "HON", "UPS", "MMM", "CAT", "UNP", "GE", "FDX", "RTX", "LMT", 
            "AAPL", "MSFT", "GOOGL", "META", "NVDA", "INTC", "ADBE", "CRM", "IBM", 
            "ECL", "DD", "LIN", "SHW", "APD", "NEM", "FCX", "IP", "MON", 
            "AMT", "PLD", "SPG", "CCI", "EQR", "PSA", "AVB", "O", "DLR", 
            "NEE", "DUK", "D", "SO", "AEP", "EXC", "ED", "SRE", "PEG"]

#### Download 6 years of stock data from Yahoo Finance

In [53]:
data = yf.download(tickers, start="2018-01-01", end="2024-01-01")["Adj Close"]

data.to_csv("financial_data.csv")

[*********************100%%**********************]  100 of 100 completed

2 Failed downloads:
['BRK.B', 'MON']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [54]:
data.shape

(1509, 100)

#### Check if 'tickers' list has duplicates

In [55]:
if(len(set(tickers)) == len(tickers)):
    print(f'Tickers list is unique')
else:
    print(f'Tickers list contains duplicate values of stock names')

Tickers list contains duplicate values of stock names


#### List has duplicate values, hence removing duplicate values

In [56]:
tickers = list(set(tickers))

In [57]:
len(tickers)

100

#### Check Five-Point Statistics of the data

In [58]:
data.describe()

Ticker,AAPL,ABT,ADBE,AEP,AMGN,AMT,AMZN,APD,AVB,AXP,...,TMO,TSLA,UNH,UNP,UPS,V,VZ,WFC,WMT,XOM
count,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,...,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000,1509.000000
mean,108.567105,91.710359,392.130418,74.832815,203.632663,198.575749,120.070930,225.242965,169.385367,128.599417,...,413.622344,145.981500,353.847944,178.748087,135.324714,188.824978,42.547618,40.573482,37.144339,66.465159
std,51.291660,21.218399,126.950048,10.687191,34.899115,41.626715,33.408151,51.794979,27.092292,31.271441,...,138.322416,113.530191,116.096370,36.585974,39.439125,37.670209,5.442731,7.803564,9.365179,23.773315
min,34.030678,50.808949,177.699997,50.740246,138.408813,115.949120,59.450500,130.924377,104.164742,65.164047,...,190.001053,11.931333,183.959396,104.550980,74.881050,109.110764,30.161383,19.531406,20.374685,25.646790
25%,51.587616,73.983253,277.570007,69.321114,169.867264,175.138702,90.698997,187.000839,147.684540,98.783150,...,275.141602,22.271000,239.659225,148.415146,95.213593,161.552490,37.162525,39.093788,29.283985,52.804234
50%,121.288040,97.912361,373.440002,76.082649,208.877655,203.948166,114.302002,235.849518,168.919510,122.673996,...,454.896332,160.190002,323.632507,188.287048,145.321884,197.173340,44.509247,42.330891,39.720573,59.263210
75%,151.247482,108.319305,489.269989,81.282089,226.980377,229.041489,154.474503,270.057861,184.835251,157.481400,...,537.673157,242.190002,476.151947,207.091522,171.338928,218.902359,47.149155,45.414780,43.833023,85.270927
max,197.857529,135.741043,688.369995,97.879768,286.222473,283.366455,186.570496,313.349426,238.481262,192.847061,...,663.555359,409.970001,548.926025,263.802338,211.246750,261.891663,51.305046,55.813000,55.775589,118.014030


## Feature Engineering

### 1. Calculate log returns

In [93]:
log_returns = data.apply(lambda x: np.log(x) - np.log(x.shift(1)))
log_returns = log_returns.iloc[1:]

In [94]:
log_returns.head()

Ticker,AAPL,ABT,ADBE,AEP,AMGN,AMT,AMZN,APD,AVB,AXP,...,TMO,TSLA,UNH,UNP,UPS,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-03,-0.000175,0.002209,0.018621,-0.008460,0.018694,0.003889,0.012694,0.005423,0.003098,0.006146,...,0.018178,-0.010286,0.010435,0.005582,0.021914,0.009906,-0.020763,0.007664,0.008685,0.019450
2018-01-04,0.004634,-0.001699,0.011970,-0.011909,-0.004223,-0.014718,0.004466,0.003838,-0.018048,0.016496,...,0.012440,-0.008325,0.004331,-0.005435,0.006466,0.003711,0.003237,0.012430,0.000905,0.001383
2018-01-05,0.011321,0.002886,0.011504,-0.002116,0.005941,0.006426,0.016033,0.008346,-0.002007,0.002278,...,0.016992,0.006210,0.018889,0.012659,0.003061,0.023667,-0.002284,0.006716,0.005910,-0.000807
2018-01-08,-0.003722,-0.002886,-0.001620,0.008719,-0.000277,0.010619,0.014322,0.001068,0.000746,-0.009243,...,0.001629,0.060755,-0.017509,0.013792,0.012072,0.004030,-0.001716,-0.011379,0.014672,0.004486
2018-01-09,-0.000115,0.001699,0.008931,-0.011831,0.015276,-0.006783,0.004665,0.002547,-0.007598,0.003688,...,0.016536,-0.008118,0.004971,0.011126,-0.000464,-0.001929,-0.003675,0.003540,-0.012079,-0.004255


In [95]:
log_returns.shape

(1508, 100)

### 2. Calculate P/E Ratio

In [105]:
PE_ratio = [] 
PE_tickers = []

for ticker in tickers:
    PE_tickers.append(ticker)
    ticker_info = yf.Ticker(ticker).info
    if "forwardPE" in ticker_info:
        PE_ratio.append(ticker_info["forwardPE"])
    else:
        PE_ratio.append(None)
dict = {"Ticker": PE_tickers, "P/E": PE_ratio}

PE_data = pd.DataFrame(dict)

In [106]:
PE_data.head()

,Ticker,P/E
0,HD,25.121153
1,AVB,34.749060
2,BMY,7.638028
3,KMI,14.440946
4,DLR,114.317460


In [107]:
PE_data.shape

(100, 2)

### 3. Calculate Beta

In [100]:
def calc_beta(ticker):
    
    # calculate covariance
    covariance = daily_returns_individual_stocks[ticker].cov(daily_returns_sp500)

    # calculate Beta
    beta = covariance / variance_sp500

    stock.append(ticker)
    betas.append(beta)

stock = []
betas = []

# download data of S&P500
data_sp500 = yf.download("^GSPC", start="2018-01-01", end="2024-01-01")["Adj Close"]

# calculate daily returns of S&P500
daily_returns_sp500 = data_sp500.pct_change()

# calculate variance of S&P500
variance_sp500 = data_sp500.pct_change().var()

# calculate daily returns of individual stocks
daily_returns_individual_stocks = pd.DataFrame()
for ticker in tickers:
    daily_returns_individual_stocks[ticker] = data[ticker].pct_change()

for ticker in tickers:
    calc_beta(ticker)

dict = {"Ticker": stock, "Beta": betas}
betas = pd.DataFrame(dict)

[*********************100%%**********************]  1 of 1 completed


In [101]:
betas.head()

,Ticker,Beta
0,HD,1.009241
1,AVB,0.838854
2,BMY,0.542670
3,KMI,0.901937
4,DLR,0.795317


In [103]:
betas.shape

(100, 2)

## find where the rows are getting reduced

### 4. Correlation between Assets

In [108]:
correlation_matrix = data.corr()

correlation_matrix

Ticker,AAPL,ABT,ADBE,AEP,AMGN,AMT,AMZN,APD,AVB,AXP,...,TMO,TSLA,UNH,UNP,UPS,V,VZ,WFC,WMT,XOM
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,1.000000,0.876145,0.781647,0.687932,0.849034,0.593931,0.679342,0.861821,0.538235,0.868193,...,0.943034,0.922546,0.940908,0.921561,0.904807,0.903105,-0.219128,-0.016059,0.948203,0.564161
ABT,0.876145,1.000000,0.841976,0.729184,0.734814,0.828058,0.819853,0.879663,0.635024,0.803933,...,0.948906,0.895625,0.780571,0.925988,0.893170,0.881800,0.193132,-0.130526,0.854225,0.238022
ADBE,0.781647,0.841976,1.000000,0.426526,0.614059,0.719321,0.915438,0.783093,0.521751,0.662194,...,0.774117,0.796144,0.603789,0.788945,0.733428,0.832502,0.197699,-0.163111,0.766173,0.059224
AEP,0.687932,0.729184,0.426526,1.000000,0.692948,0.736274,0.356355,0.713623,0.629534,0.718611,...,0.764641,0.602299,0.732749,0.752934,0.664362,0.735448,0.072669,-0.046887,0.718683,0.444499
AMGN,0.849034,0.734814,0.614059,0.692948,1.000000,0.539600,0.514122,0.835039,0.293527,0.672664,...,0.789632,0.694992,0.857368,0.780373,0.714750,0.850513,-0.196290,-0.168098,0.889811,0.544768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V,0.903105,0.881800,0.832502,0.735448,0.850513,0.710433,0.716272,0.926984,0.542792,0.828182,...,0.870286,0.783831,0.807886,0.881365,0.807505,1.000000,-0.000973,-0.127373,0.936509,0.417088
VZ,-0.219128,0.193132,0.197699,0.072669,-0.196290,0.492939,0.384519,-0.019551,0.207953,-0.161702,...,-0.044093,-0.045584,-0.352428,0.024869,-0.048340,-0.000973,1.000000,-0.289688,-0.176263,-0.756347
WFC,-0.016059,-0.130526,-0.163111,-0.046887,-0.168098,-0.274539,-0.298394,-0.240080,0.468867,0.352220,...,-0.051197,0.039987,0.117163,0.066718,0.102177,-0.127373,-0.289688,1.000000,-0.147924,0.431260


In [109]:
type(correlation_matrix)

pandas.core.frame.DataFrame

## Regression Analysis

In [110]:
# Perform PCA (Principal Component Analysis)
pca = PCA(n_components = 1)  # Reduce to 1 component
pca.fit(correlation_matrix)

# Select the first principal component
first_principal_component = pca.components_[0]

# Project the data onto the first principal component
reduced_vector = np.dot(correlation_matrix, first_principal_component)

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
dict = {'Ticker': list((correlation_matrix.index)), 'Correlation_PCA': reduced_vector}
correlation_pca_matrix = pd.DataFrame(dict)

In [ ]:
independent_features = log_returns
independent_features = independent_features.T

In [ ]:
independent_features.reset_index(inplace=True)  # Reset index and make the current index a normal column
independent_features.rename(columns={'index': 'Ticker'}, inplace=True)  # Rename the old index column if needed
independent_features.index = range(1, len(independent_features) + 1)  # Set new index column with serial numbers starting from 1
independent_features.head()

#### Inner Joins

In [ ]:
independent_features = pd.merge(independent_features, PE_data, how ='inner', on ='Ticker')
independent_features = pd.merge(independent_features, betas, how ='inner', on ='Ticker')
independent_features = pd.merge(independent_features, correlation_pca_matrix, how ='inner', on ='Ticker')

In [ ]:
independent_features.head()

In [ ]:
X = independent_features
Y = daily_returns_individual_stocks.T

X

In [ ]:
Y

In [ ]:
X = independent_features
Y = daily_returns_individual_stocks.T

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model (for demonstration purposes)
score = model.score(X_test, y_test)
print("R-squared score:", score)

# X = sm.add_constant(X)  # Add constant term
# model = sm.OLS(y, X).fit()  # Fit regression model
# print(model.summary())  # Display regression summary